In [8]:
model_file = 'resnet50'
url, imgFile = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
url, classFile = ("https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt", "imagenet_classes.txt")

In [ ]:
# Download an example image from the pytorch website
import urllib
try: urllib.URLopener().retrieve(url, imgFile)
except: urllib.request.urlretrieve(url, imgFile)

# Download ImageNet labels
import urllib
try: urllib.URLopener().retrieve(url, classFile)
except: urllib.request.urlretrieve(url, classFile)

In [ ]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', model_file, pretrained=True)
# or any of these variants
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)
model.eval()
torch.save(model, model_file + '.pth')

In [ ]:
import torch

device = torch.device('cpu')

model = torch.load(model_file + '.pth')
model = model.eval().to(device)

input = torch.randn((1, 3, 224, 224), dtype=torch.float32).to(device)
input = input.to(device)

dynamic_axes = {
    "input": {0: "batch_size"},
    "output": {0: "batch_size"}
}

torch.onnx.export(
    model,
    input,
    model_file + ".onnx",
    export_params=True,
    do_constant_folding=True,
    input_names=["input"],
    output_names=["output"],
    dynamic_axes=dynamic_axes,
    opset_version=11,
)

print("onnx model export success")

In [1]:
!python -m onnxruntime.tools.make_dynamic_shape_fixed --dim_param batch_size --dim_value 1 "resnet50.onnx" "resnet50_shape1.onnx"

In [ ]:
import onnxruntime
from pathlib import Path
import os

options = onnxruntime.SessionOptions()  
model_path = Path(model_file + '_shape1.onnx')
onnx_ctx_path = model_path.with_suffix('.onnx_ctx.onnx')  
print(onnx_ctx_path)
if os.path.exists(onnx_ctx_path):      
    model_path = onnx_ctx_path    
else:      
    options.add_session_config_entry('ep.context_enable', '1')      
    options.add_session_config_entry('ep.context_embed_mode', '0')
# (Optional) Enable configuration that raises an exception if the model can't be
# run entirely on the QNN HTP backend.
options.add_session_config_entry("session.disable_cpu_ep_fallback", "1")

ort_session = onnxruntime.InferenceSession(model_path, 
                                       sess_options=options,
                                       providers=["QNNExecutionProvider"],
                                       provider_options=[{"backend_path": "QnnHtp.dll"}])
# ort_session = onnxruntime.InferenceSession(model_file + '.onnx', providers=['CPUExecutionProvider'])
input_name = ort_session.get_inputs()[0].name
output_name = ort_session.get_outputs()[0].name
print(input_name, output_name)
print("Available providers:", ort_session.get_providers())
print("Current provider:", ort_session.get_provider_options())

In [ ]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
import torch.nn.functional as F

input_image = Image.open(imgFile)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0).numpy() # create a mini-batch as expected by the model


print(input_tensor.shape)
print(input_batch.shape)


In [ ]:
import torch
for i in range(1000): #Inferencing 1000 times makes the NPU run for a few seconds.
    pred_logits = ort_session.run([output_name], {input_name: input_batch})[0]
    pred_logits = torch.tensor(pred_logits)
    pred_softmax = F.softmax(pred_logits, dim=1)
    top5_prob, top5_catid = torch.topk(pred_softmax, 5)
    top5_prob = top5_prob[0]
    top5_catid = top5_catid[0]

# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())